In [ ]:
import cv2
import numpy as np
import time

import mediapipe as mp

In [ ]:
# # functions
# def prepare_camera():
#     return cv2.VideoCapture(0)


# def get_hand_data():
#     imgs = []
#     r_imgs = []
#     taking = False
#     n_images = 100
#     last_time = -1
    
#     cam = prepare_camera()
#     cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1000)  # Set width
#     cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 1000)  # Set height
#     W = int(cam.get(cv2.CAP_PROP_FRAME_WIDTH))
#     H = int(cam.get(cv2.CAP_PROP_FRAME_HEIGHT))
#     print(f"Camera resolution: {W}x{H}")
#     w = h = 300
#     i = 0
    
#     pos = np.array([(int(W*i)-w//2,int(H*j)-h//2) for i,j in [(.5,.5),(.25,.25),(.25,.75),(.75,.25),(.75,.75)]])
#     cv2.namedWindow("image_capture")
    
#     while i != len(pos):
#         ret, frame = cam.read()
#         cv2.rectangle(frame,pos[i],pos[i]+[w,h],(0, 0,255),2)
#         cv2.imshow("test", frame)
#         k = cv2.waitKey(20)
#         if taking:
#             if (dateT.now() - last_time).total_seconds() <= 0.2:
#                 continue
#             r_imgs.append(frame)
#             last_time = dateT.now()
#             # print(f"{len(r_imgs)}/{n_images}")
#             if len(r_imgs) != n_images:
#                 continue
#             taking = False
#             i+= 1
#             imgs += r_imgs
#             r_imgs = []
            
#         elif k == 32: #space
#             print("Taking photos")
#             taking = True
#             last_time = dateT.now()
            
#         elif k == 27: #ESC key
#             break
#     cam.release()
#     cv2.destroyAllWindows()
#     return imgs


In [ ]:
class landmarker_and_result():
    def __init__(self):
        self.result = mp.tasks.vision.HandLandmarkerResult
        self.landmarker = mp.tasks.vision.HandLandmarker
        self.createLandmarker()

    def createLandmarker(self):
        # callback function
        def update_result(result: mp.tasks.vision.HandLandmarkerResult, output_image: mp.Image, timestamp_ms: int):
            self.result = result

        # HandLandmarkerOptions (details here: https://developers.google.com/mediapipe/solutions/vision/hand_landmarker/python#live-stream)
        options = mp.tasks.vision.HandLandmarkerOptions( 
            base_options = mp.tasks.BaseOptions(model_asset_path="hand_landmarker.task"), # path to model
            running_mode = mp.tasks.vision.RunningMode.LIVE_STREAM, # running on a live stream
            num_hands = 2, # track both hands
            min_hand_detection_confidence = 0.3, # lower than value to get predictions more often
            min_hand_presence_confidence = 0.3, # lower than value to get predictions more often
            min_tracking_confidence = 0.3, # lower than value to get predictions more often
            result_callback=update_result)
        
        # initialize landmarker
        self.landmarker = self.landmarker.create_from_options(options)

    def detect_async(self, frame):
        # convert np frame to mp image
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=frame)
        # detect landmarks
        self.landmarker.detect_async(image = mp_image, timestamp_ms = int(time.time() * 1000))

    def close(self):
        # close landmarker
        self.landmarker.close()

def draw_landmarks_on_image(rgb_image, detection_result: mp.tasks.vision.HandLandmarkerResult):
    """Courtesy of https://github.com/googlesamples/mediapipe/blob/main/examples/hand_landmarker/python/hand_landmarker.ipynb"""
    try:
        if detection_result.hand_landmarks == []:
            return rgb_image
        else:
            hand_landmarks_list = detection_result.hand_landmarks
            annotated_image = np.copy(rgb_image)
            H,W = annotated_image.shape[:2]
            # print(W,H)

            # Loop through the detected hands to visualize.
            for idx in range(len(hand_landmarks_list)):
                hand_landmarks = hand_landmarks_list[idx]
                x_coordinates = [landmark.x for landmark in hand_landmarks]
                y_coordinates = [landmark.y for landmark in hand_landmarks]
                cv2.rectangle(annotated_image, 
                              (int(min(x_coordinates)*W),int(min(y_coordinates)*H)), 
                              (int(max(x_coordinates)*W),int(max(y_coordinates)*H)),
                              (0,0,255),2)
                # print(min(x_coordinates),max(x_coordinates))
                
                # # Draw the hand landmarks.
                # hand_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
                # hand_landmarks_proto.landmark.extend([
                #    landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in hand_landmarks])
                # mp.solutions.drawing_utils.draw_landmarks(
                #    annotated_image,
                #    hand_landmarks_proto,
                #    mp.solutions.hands.HAND_CONNECTIONS,
                #    mp.solutions.drawing_styles.get_default_hand_landmarks_style(),
                #    mp.solutions.drawing_styles.get_default_hand_connections_style())
            return annotated_image
    except:
        return rgb_image

# create landmarker
hand_landmarker = landmarker_and_result()

def main():
    # access webcam
    cap = cv2.VideoCapture(0)
    while True:
        # pull frame
        ret, frame = cap.read()
        # mirror frame
        frame = cv2.flip(frame, 1)
        # update landmarker results
        hand_landmarker.detect_async(frame)
        # draw landmarks on frame
        frame = draw_landmarks_on_image(frame,hand_landmarker.result)
        # display frame
        cv2.imshow('frame',frame)
        if cv2.waitKey(20) == ord('q'):
            break

    # release everything
    cap.release()
    cv2.destroyAllWindows()
    hand_landmarker.close()
main()
